<a href="https://colab.research.google.com/github/TwistedAlex/TensorFlow2Practice/blob/main/TF_TUTORIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Eager execution**
Evaluate operations immediately, without building graphs: operations *return concrete values* instead of constructing a compuational graph to run later. 
  *An intuitive interface*: Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.
  *Easier debugging*: Call ops directly to inspect running models and test changes. Use standard Python debugging toools for immediate error report.
  *Natural control flow*: Use Python control flow instead of graph control flow, simplifying the specification of dynamic models.

**Setup and basic usage**

In [1]:
import os
import tensorflow as tf
import cProfile
tf.compat.v1.enable_eager_execution()

In [2]:
# Eager execution is enabled by default in TF2.0

tf.executing_eagerly()
# Run TF ops and the results will return immediately:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))
a = tf.constant([[1, 2], [3, 4]])
print(a)

hello, [[4.]]
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


Broadcasting

In [3]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


Operator overloading

In [4]:
print(a * b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


Use NumPy values

In [5]:
import numpy as np

c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


Obtain numpy value from a tensor:

In [7]:
print(a.numpy())

[[1 2]
 [3 4]]


**Dynamic control flow**

A major benefit of eager execution is that all the functionality of the host language is available while your model is executing. So, for example, it is easy to write fizzbuzz:

In [8]:
def fizzbuzz(max_num):
  counter = tf.constant(0)
  max_num = tf.convert_to_tensor(max_num)
  for num in range(1, max_num.numpy()+1):
    num = tf.constant(num)
    if int(num % 3) == 0 and int(num % 5) == 0:
      print('FizzBuzz')
    elif int(num % 3) == 0:
      print('Fizz')
    elif int(num % 5) == 0:
      print('Buzz')
    else:
      print(num.numpy())
    counter += 1

This has conditionals that depend on tensor values and it prints these values at runtime.

In [9]:
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


**Eager training**

Computing gradients

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks. During eager execution, use tf.GradientTape to trace operations for computing gradients later.

Disable eager execution

In [3]:
tf.compat.v1.disable_eager_execution()
a = tf.constant([[1, 2], [3, 4]])
print(a)

Tensor("Const:0", shape=(2, 2), dtype=int32)


In [4]:
b = tf.add(a, 1)
print(b)

Tensor("Add:0", shape=(2, 2), dtype=int32)


In [ ]:
import numpy as np

c = np.multiply(a, b)
print(c)